In [1]:

# ----------------------
# CELL 1: Imports & setup
# ----------------------
import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
from io import BytesIO
from PIL import Image
import base64

# Load environment variables
load_dotenv(override=True)

# Ollama local settings
OLLAMA_BASE_URL = "http://localhost:11434/v1"
openai = OpenAI(base_url=OLLAMA_BASE_URL, api_key="ollama")

# System prompt
SYSTEM_PROMPT = """
You are a Technical Question Answering AI. You answer Python programming questions clearly and concisely.
Provide explanations, examples, and reasoning. Keep answers beginner-friendly when possible.
"""

# Optional: Tool to show Python code examples
def run_code_example(code: str):
    return f"Code example:\n{code}"

tools = [
    {
        "type": "function",
        "function": {
            "name": "run_code_example",
            "description": "Return Python code examples for explanations",
            "parameters": {
                "type": "object",
                "properties": {
                    "code": {"type": "string", "description": "Python code to illustrate the explanation"}
                },
                "required": ["code"]
            }
        }
    }
]

In [2]:

# ----------------------
# CELL 2: Chat & message functions
# ----------------------
def chat(message, history, selected_model):
    # Format the history for Ollama
    history = [{"role": h["role"], "content": h["content"]} for h in history]
    messages = [{"role": "system", "content": SYSTEM_PROMPT}] + history + [{"role": "user", "content": message}]

    # Streaming response from Ollama
    stream = openai.chat.completions.create(
        model=selected_model,  # <-- model switched dynamically
        messages=messages,
        stream=True,
        tools=tools
    )

    response_text = ""
    for chunk in stream:
        response_text += chunk.choices[0].delta.content or ""
        yield response_text

def put_message_in_chatbot(message, history):
    return "", history + [{"role": "user", "content": message}]

In [3]:

# ----------------------
# CELL 3: Gradio UI
# ----------------------
with gr.Blocks(css="""
    body { background-color: #FFFFFF; font-family: 'Arial', sans-serif; }
    .gradio-container { background-color: #FFFFFF; }

    .gradio-chatbot {
        background-color: #C0392B;
        color: #FFFFFF; 
        font-weight: bold;
        border-radius: 15px;
        padding: 12px;
    }

    .gradio-chatbot .message.user {
        background-color: #E74C3C;
        color: #FFFFFF;
        font-weight: bold;
        border-radius: 12px;
        padding: 10px;
    }

    .gradio-chatbot .message.assistant {
        background-color: #A93226;
        color: #FFFFFF;
        font-weight: bold;
        border-radius: 12px;
        padding: 10px;
    }

    .gradio-textbox textarea {
        background-color: #F1948A;
        color: #000000; 
        font-weight: bold;
        border-radius: 12px;
        padding: 8px;
    }

    .gradio-button {
        background-color: #E74C3C; 
        color: #FFFFFF; 
        font-weight: bold;
        border-radius: 10px;
        padding: 8px 16px;
    }
""") as ui:

    # Model selection dropdown
    model_selector = gr.Dropdown(
        choices=["llama3.2", "phi3", "mistral"],
        value="llama3.2",
        label="Select Model"
    )

    chatbot = gr.Chatbot(height=500, type="messages")
    message = gr.Textbox(label="Ask your Python technical question:")

    # Submit chain
    message.submit(
        put_message_in_chatbot,
        inputs=[message, chatbot],
        outputs=[message, chatbot]
    ).then(
        chat,
        inputs=[message, chatbot, model_selector],  # <-- pass selected model
        outputs=chatbot
    )

# Launch the Gradio interface
ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/opt/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/queueing.py", line 626, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/route_utils.py", line 350, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/blocks.py", line 2235, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/blocks.py", line 1758, in call_function
    prediction = await utils.async_iteration(iterator)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/utils.py", line 762, in async_iteration
    return await anext(it